In [1]:
CONFIG = {
    'data_folder': 'C:/Users/Николай/PycharmProjects/VKRecSys/data/',
    'models_folder': 'C:/Users/Николай/PycharmProjects/VKRecSys/B.Processing/Модели/',
    'results_folder': 'C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/',

    'test_path': 'test_pairs.csv',  
    'model_path': '6.14_2.pth', 
    'output_path': '6.14_2_predictions.csv',
    'items_meta_path' : 'items_meta.parquet',
    'users_meta_path' : 'users_meta.parquet',

    'user_emb_size' : 256, # 183404
    'item_emb_size' : 256, # 337727
    'source_emb_size' : 256, # 19613
    'age_emb_size' : 256, # 43
    'duration_emb_size' : 256, # ~175   
    'gender_emb_size' : 256, # 3
    'torch_precision' : 40, # number of decimal places for printing numbers

    'DEVICE': 'cuda',
    'SEED': 42,
    'BATCH_SIZE': 16384,
    'output_dim': 3
}

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm

In [3]:
device = torch.device(CONFIG['DEVICE'] if torch.cuda.is_available() else "cpu")
torch.set_printoptions(precision=CONFIG['torch_precision'])

torch.manual_seed(CONFIG['SEED'])
torch.cuda.manual_seed_all(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])

In [4]:
test = pd.read_csv(f"{CONFIG['data_folder']}{CONFIG['test_path']}")

items_meta = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['items_meta_path']}", engine='pyarrow')
items_meta['duration'] = items_meta['duration'] - 5
items_meta['item_id'] = items_meta['item_id'].astype('category')
items_meta['source_id'] = items_meta['source_id'].astype('category')
items_meta['duration'] = items_meta['duration'].astype('category')
items_meta.set_index('item_id', inplace=True)

users_meta = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['users_meta_path']}", engine='pyarrow')
users_meta['age'] = users_meta['age'] - 18
users_meta['gender'] = users_meta['gender'].replace({1:0, 2:1})
users_meta['user_id'] = users_meta['user_id'].astype('category')
users_meta['gender'] = users_meta['gender'].astype('category')
users_meta['age'] = users_meta['age'].astype('category')
users_meta.set_index('user_id', inplace=True)

In [5]:
class Model(nn.Module):
    def __init__(self, 
                 input_dim, 
                 num_users=users_meta.index.nunique(), 
                 num_items=items_meta.index.nunique(), 
                 num_sources=items_meta['source_id'].nunique(),
                 num_ages=users_meta['age'].nunique(),
                 num_durations=items_meta['duration'].nunique(),
                 num_genders=users_meta['gender'].nunique(), 
                 output_dim=CONFIG['output_dim']): 
        super(Model, self).__init__()
        
        # Embedding layers for each feature
        self.user_embedding = nn.Embedding(num_users, CONFIG['user_emb_size'])
        self.item_embedding = nn.Embedding(num_items, CONFIG['item_emb_size'])
        self.source_embedding = nn.Embedding(num_sources, CONFIG['source_emb_size'])
        self.age_embedding = nn.Embedding(num_ages, CONFIG['age_emb_size'])
        self.duration_embedding = nn.Embedding(num_durations, CONFIG['duration_emb_size'])
        self.gender_embedding = nn.Embedding(num_genders, CONFIG['gender_emb_size'])
        
        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 256)
        self.fc6 = nn.Linear(256, 128)
        self.fc7 = nn.Linear(128, 128)
        self.fc8 = nn.Linear(128, 64)
        self.target_output = nn.Linear(64, output_dim)
        self.share_output = nn.Linear(64, 1)
        self.bookmark_output = nn.Linear(64, 1)
        self.timespent_output = nn.Linear(64, 255)
        
        self.gelu = nn.GELU()
    
    def forward(self, user_ids, item_ids, source_ids, age_ids, duration_ids, gender_ids, embeddings):
        
        # Obtain embeddings for all features
        user_emb = self.user_embedding(user_ids)
        item_emb = self.item_embedding(item_ids)
        source_emb = self.source_embedding(source_ids)
        age_emb = self.age_embedding(age_ids)
        duration_emb = self.duration_embedding(duration_ids)
        gender_emb = self.gender_embedding(gender_ids)
        
        x = torch.cat((user_emb, item_emb, source_emb, age_emb, duration_emb, gender_emb, embeddings), dim=1)
        
        x = self.gelu(self.fc1(x))
        
        x = self.gelu(self.fc2(x))
        
        x = self.gelu(self.fc3(x))
        
        x = self.gelu(self.fc4(x))
        
        x = self.gelu(self.fc5(x))
        
        x = self.gelu(self.fc6(x))
        
        x = self.gelu(self.fc7(x))
        
        x = self.gelu(self.fc8(x))
        
        target_output = self.target_output(x)
        share_output = self.share_output(x)
        bookmark_output = self.bookmark_output(x)
        timespent_output = self.timespent_output(x)
        
        return target_output, share_output, bookmark_output, timespent_output

In [6]:
input_dim = (CONFIG['user_emb_size'] + 
             CONFIG['item_emb_size'] + 
             CONFIG['source_emb_size'] + 
             CONFIG['age_emb_size'] +
             CONFIG['duration_emb_size'] + 
             CONFIG['gender_emb_size'] + 
             32)

In [7]:
model = Model(input_dim).to(device)
model.load_state_dict(torch.load(f"{CONFIG['models_folder']}{CONFIG['model_path']}")['model_state_dict'])
model.eval()

C:\Users\Николай\AppData\Local\Temp\ipykernel_21648\2686595169.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{CONFIG['models_folder'

Model(
  (user_embedding): Embedding(183404, 256)
  (item_embedding): Embedding(337727, 256)
  (source_embedding): Embedding(19613, 256)
  (age_embedding): Embedding(43, 256)
  (duration_embedding): Embedding(176, 256)
  (gender_embedding): Embedding(2, 256)
  (fc1): Linear(in_features=1568, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=256, bias=True)
  (fc6): Linear(in_features=256, out_features=128, bias=True)
  (fc7): Linear(in_features=128, out_features=128, bias=True)
  (fc8): Linear(in_features=128, out_features=64, bias=True)
  (target_output): Linear(in_features=64, out_features=3, bias=True)
  (share_output): Linear(in_features=64, out_features=1, bias=True)
  (bookmark_output): Linear(in_features=64, out_features=1, bias=True)
  (timespent_output): Linear(in_f

In [8]:
predictions = []
num_samples = len(test)
num_batches = (num_samples + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']

with torch.no_grad():
    for batch_idx in tqdm(range(num_batches), desc="Predicting", unit="batch"):
        start_idx = batch_idx * CONFIG['BATCH_SIZE']
        end_idx = min(start_idx + CONFIG['BATCH_SIZE'], num_samples)
        batch_main = test.iloc[start_idx:end_idx]
        batch_user_ids = batch_main['user_id'].values
        batch_item_ids = batch_main['item_id'].values
        
        batch_users_meta = users_meta.loc[batch_user_ids]
        batch_items_meta = items_meta.loc[batch_item_ids]
        
        batch_user_values = torch.tensor(batch_user_ids, dtype=torch.long, device=device)
        batch_item_values = torch.tensor(batch_item_ids, dtype=torch.long, device=device)
        
        batch_gender_values = torch.tensor(batch_users_meta['gender'].values, dtype=torch.long, device=device)
        batch_age_values = torch.tensor(batch_users_meta['age'].values, dtype=torch.long, device=device)
        batch_source_values = torch.tensor(batch_items_meta['source_id'].values, dtype=torch.long, device=device)
        batch_duration_values = torch.tensor(batch_items_meta['duration'].values, dtype=torch.long, device=device)

        embeddings = torch.tensor(np.stack(batch_items_meta['embeddings'].values), device=device, dtype=torch.float32)

        target_outputs, share_outputs, bookmark_outputs, timespent_outputs = model(batch_user_values,
                                                                                       batch_item_values, 
                                                                                       batch_source_values,
                                                                                       batch_age_values, 
                                                                                       batch_duration_values, 
                                                                                       batch_gender_values, 
                                                                                       embeddings)
        probabilities = F.softmax(target_outputs, dim=1)

        # Взвешенные предсказания
        class_weights = torch.tensor([0, 1, 2], device=probabilities.device, dtype=probabilities.dtype)
        weighted_predictions = torch.sum(probabilities * class_weights, dim=1).cpu().numpy()

        predictions.extend(weighted_predictions)

Predicting: 100%|██████████| 102/102 [00:09<00:00, 10.26batch/s]


In [9]:
# Добавление предсказанных значений в DataFrame
test['predict'] = predictions

test.to_csv(f"{CONFIG['results_folder']}{CONFIG['output_path']}", index=False)
print(f"Predictions saved to f'{CONFIG['results_folder']}{CONFIG['output_path']}'")

Predictions saved to f'C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/6.14_2_predictions.csv'
